<a href="https://colab.research.google.com/github/AlexUmnov/LLM-Engineering-Essentials/blob/main/topic2/2.2_llm_workflows_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Engineering Essentials by Nebius Academy

Course github: [link](https://github.com/Nebius-Academy/LLM-Engineering-Essentials/tree/main)

# 2.2. Structured Inputs and Output

# **Practice task solutions**

In [2]:
!pip install openai -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.3/683.3 kB 16.6 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
from openai import OpenAI
import os

os.environ['NEBIUS_API_KEY'] = userdata.get("nebius_api_key")

nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llama_model = "meta-llama/Llama-3.3-70B-Instruct"

def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt="You are a helpful assistant",
                    max_tokens=512,
                    client=nebius_client,
                    model=llama_model,
                    prettify=True,
                    temperature=None) -> str:

    messages = []

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content


##  Task 1. Character localization

Let's add localization to our simple chat NPC class from Topic 1.

Your task will be to implement the following localized chat pipeline:
- The user's input is translated into English,
- The NPC answers an English query in English (already implemented)
- The NPC's answer is translated into the target language, and the translation is returned to the user.

In [4]:
# Class copypaste
import os

#with open("nebius_api_key", "r") as file:
#    nebius_api_key = file.read().strip()

#os.environ["NEBIUS_API_KEY"] = nebius_api_key
from google.colab import userdata
os.environ["NEBIUS_API_KEY"] = userdata.get("nebius_api_key")

from collections import defaultdict, deque
from openai import OpenAI
from typing import Dict, Any, Optional
import datetime
import string
import random
from dataclasses import dataclass

@dataclass
class NPCConfig:
    world_description: str
    character_description: str
    history_size: int = 10
    has_scratchpad: bool = False

class NPCFactoryError(Exception):
    """Base exception class for NPC Factory errors."""
    pass

class NPCNotFoundError(NPCFactoryError):
    """Raised when trying to interact with a non-existent NPC."""
    def __init__(self, npc_id: str):
        self.npc_id = npc_id
        super().__init__(f"NPC with ID '{npc_id}' not found")

class SimpleChatNPC:
    def __init__(self, client: OpenAI, model: str, config: NPCConfig):
        self.client = client
        self.model = model
        self.config = config
        self.chat_histories = defaultdict(lambda: deque(maxlen=config.history_size))

    def localize_input(self, input_text: str) -> str:
        """Translate user input into English."""
        return self.client.chat.completions.create(
            messages=[
                {
                    'role': 'user',
                    'content': f'Translate the following text into English:\n{input_text}. Only output the translation itself.'
                }
            ],
            model=self.model,
        ).choices[0].message.content

    def localize_output(self, output_text: str, target_language: str) -> str:
        """Translate the output into the target language."""
        return self.client.chat.completions.create(
            messages=[
                {
                    'role': 'user',
                    'content': f'Translate the following text into {target_language}:\n{output_text}. Only output the translation itself.'
                }
            ],
            model=self.model,
        ).choices[0].message.content

    def get_system_message(self) -> Dict[str, str]:
        """Returns the system message that defines the NPC's behavior."""
        character_description = self.config.character_description

        if self.config.has_scratchpad:
            character_description += """
You can use scratchpad for thinking before you answer: whatever you output between #SCRATCHPAD and #ANSWER won't be shown to anyone.
You start your output with #SCRATCHPAD and after you've done thinking, you #ANSWER"""

        return {
            "role": "system",
            "content": f"""WORLD SETTING: {self.config.world_description}
###
{character_description}"""
        }

    def chat(self, user_message: str, user_id: str, user_language: str) -> str:
        """Process a user message and return the NPC's response."""
        messages = [self.get_system_message()]

        # Add conversation history
        history = list(self.chat_histories[user_id])
        if history:
            messages.extend(history)

        # Add new user message
        if user_language != "English":
            user_message_dict = {
                "role": "user",
                "content": self.localize_input(user_message)
            }
        else:
            user_message_dict = {
                "role": "user",
                "content": user_message
            }
        self.chat_histories[user_id].append(user_message_dict)
        messages.append(user_message_dict)

        try:
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.6
            )

            response = completion.choices[0].message.content

            # Handle scratchpad if enabled
            response_clean = response
            if self.config.has_scratchpad:
                import re
                scratchpad_match = re.search(r"#SCRATCHPAD(:?)(.*?)#ANSWER(:?)", response, re.DOTALL)
                if scratchpad_match:
                    response_clean = response[scratchpad_match.end():].strip()

            if user_language != "English":
                response_clean = self.localize_output(response_clean, user_language)

            # Store response in history, including the scratchpad
            self.chat_histories[user_id].append({
                "role": "assistant",
                "content": response
            })

            # Return the message to the user without a scratchpad
            return response_clean

        except Exception as e:
            return f"Error: {str(e)}"

class NPCFactory:
    def __init__(self, client: OpenAI, model: str):
        self.client = client
        self.model = model
        self.npcs: Dict[str, SimpleChatNPC] = {}
        self.user_ids: Dict[str, str] = {}  # username -> user_id mapping
        self.user_preffered_languages: Dict[str, str] = {}  # user_id -> language mapping

    def set_user_language(self, user_id: str, language: str):
        """Set the preferred language for a user."""
        self.user_preffered_languages[user_id] = language

    def generate_id(self) -> str:
        """Generate a random unique identifier."""
        return ''.join(random.choice(string.ascii_letters) for _ in range(8))

    def register_user(self, username: str) -> str:
        """Register a new user and return their unique ID.
        If username already exists, appends a numerical suffix."""
        base_username = username
        suffix = 1

        # Keep trying with incremented suffixes until we find an unused name
        while username in self.user_ids:
            username = f"{base_username}_{suffix}"
            suffix += 1

        user_id = self.generate_id()
        self.user_ids[username] = user_id
        return user_id

    def register_npc(self, world_description: str, character_description: str,
                     history_size: int = 10, has_scratchpad: bool = False) -> str:
        """Create and register a new NPC, returning its unique ID."""
        npc_id = self.generate_id()

        config = NPCConfig(
            world_description=world_description,
            character_description=character_description,
            history_size=history_size,
            has_scratchpad=has_scratchpad
        )

        self.npcs[npc_id] = SimpleChatNPC(self.client, self.model, config)
        return npc_id

    def chat_with_npc(self, npc_id: str, user_id: str, message: str) -> str:
        """Send a message to a specific NPC from a specific user.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user
            message: The message to send

        Returns:
            The NPC's response

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        user_language = self.user_preffered_languages.get(user_id, "English")
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        npc = self.npcs[npc_id]
        return npc.chat(message, user_id, user_language=self.user_preffered_languages[user_id])

    def get_npc_chat_history(self, npc_id: str, user_id: str) -> list:
        """Retrieve chat history between a specific user and NPC.

        Args:
            npc_id: The unique identifier of the NPC
            user_id: The unique identifier of the user

        Returns:
            List of message dictionaries containing the chat history

        Raises:
            NPCNotFoundError: If the specified NPC doesn't exist
        """
        if npc_id not in self.npcs:
            raise NPCNotFoundError(npc_id)

        return list(self.npcs[npc_id].chat_histories[user_id])

In [5]:
from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

model = "meta-llama/Meta-Llama-3.1-405B-Instruct"

# Creating a factory
npc_factory = NPCFactory(client=client, model=model)

In [6]:
# Register a user
user_id = npc_factory.register_user("Alice")

# Set user preffered language
preffered_language = "Old English"
npc_factory.set_user_language(user_id, preffered_language)

# Create an NPC
npc_id = npc_factory.register_npc(
    world_description="Medieval London, XIII century",
    character_description="A knight at Edward I's court",
    has_scratchpad=False
)

In [7]:
def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n")
    for line in lines:
        current_line = ""
        words = line.split()
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

In [8]:
# We can hack our own code a bit and use translation features to test the system.

npc = npc_factory.npcs[npc_id]

message = "Hello, who are you and what brings you here?"
message_translated = npc.localize_output(message, preffered_language)
print(f"Translated message: {message_translated}")

response = npc_factory.chat_with_npc(
    npc_id,
    user_id,
    message_translated
)
print("Original answer")
print(prettify_string(response))
print("Translated answer")
print(prettify_string(npc.localize_input(response)))

Translated message: "Hailo, hwā eart þū and hwæt bringeþ þē hēr?"
Original answer
"Godes bletsunge si þe Kyng, gode syr. Ic eom Syr Eadweard de Montfort, eadmod
cniht on þeowan to ure miclan cyning, Eadweard I. Ic eom cuman to attendantenne
þe cyninges heall, swa swa is min pliht, and to offre mine ræd on þingum rycst
and gewinne, gyf hit si bidden of me. Hit is wyrðo to þeowan þe Kyng, and ic
eom stol on stodan betwixo þa æþelan cnihtas þe ure ryc weriað. Ic bidde, secga
me, hwæt þu sy, þæt ic witan mote hwem ic spece.".
Translated answer
"God's blessings be upon the King, good sir. I am Sir Edward de Montfort, a
humble knight at the service of our great king, Edward I. I have come to attend
the king's hall, as is my duty, and to offer my counsel on matters of power and
war, if it be asked of me. It is an honor to serve the King, and I am proud to
stand among the noble knights who defend our realm. I pray you, tell me, who
are you, that I may know to whom I speak."


## Task 2. Translating poetry

LLMs are notoriously bad at translating poetry. The resulting poems rarely have good rhyme and rhytm. Let's try to naively translate some Humpty Dumpty rhyme to a language of your choice:

In [9]:
language = "Dutch"
poem = """Humpty Dumpty sat on a wall,
Humpty Dumpty had a great fall.
All the king's horses and all the king's men
Couldn't put Humpty together again."""

print(answer_with_llm(
    f"Translate the following children's rhyme to {language}\n{poem}"
))

Here's the translation of the rhyme to Dutch:

Humpty Dumpty zat op een muur,
Humpty Dumpty had een grote val.
Alle paarden van de koning en alle mannen van de koning
Konden Humpty niet meer in elkaar zetten.

Note: In some Dutch-speaking countries, the name "Humpty Dumpty" is translated
to "Hompie Dompie" or "Hompie Dompie", but I've kept the original name in the
translation to maintain consistency.


This is hardly a good translation because the rhyme is completely broken.

You task is to create a chain of LLM calls to do the following steps in translating a poem:

1. Do a naive literal translation to preserve the meaning
2. Rewrite the translation to retain rhyme and rhythm but perhaps loosing a bit of meaning.
3. Finally have an editor look at both original and translation and make final touch ups.

We encourage you to try and prompt your LLMs to do the job of "Translator", "Editor" and so on.
Also choose the language you can understand best so that you can evaluate the result well (you can also change the original to some other language)

In [10]:
def translate_in_stages(input: str, language: str) -> str:
    pass

In [11]:
literal_translator_system_prompt = f"""
You are a translator, whose task is to translate whatever you receive literally to {language}.
You don't care if the text changes sentences, length, anything else, you only task is to retain as much meaning as possible
and be as literal as possible in your translation.
Output 'Translation:' and then the translation you created.
"""

rhyme_rewriter_system_prompt = f"""
You are a rhyme writer. You task is to receive a text in {language} and rewrite
it in a rhymed way also in {language}. It should have rhymes following one of the popular patterns,
for example every other line, two and two and so on.
You can distort the meaning a bit but not to lose it completely.
Try to not make the text much longer.
Output 'Rewriting:' and then the rewriting you created.
"""

editor_system_prompt = f"""
You are an editor.
You task is to inspect the the original text, naive translation and naive rewriting.
Assess the quality of all of them and make finishing touch ups on the rhymed rewriting.
Do not the text much longer than the original, it should have the same amount of lines.
Do not output anything after the translation.
Output the remarks you have for it in English and then
Output 'Final translation:' and the final version of the translation to {language}
"""


def translate_in_stages(input: str, language: str) -> str:
    naive_translation = answer_with_llm(
        system_prompt=literal_translator_system_prompt,
        prompt=input
    )
    print(f"\n\n{naive_translation}\n\n")

    rhymed_text = answer_with_llm(
        system_prompt=rhyme_rewriter_system_prompt,
        prompt=naive_translation
    )
    print(f"\n\n{rhymed_text}\n\n")

    editor_notes = answer_with_llm(
        system_prompt=editor_system_prompt,
        prompt=f"""
Original_text: {input}
Naive translation: {naive_translation}
Rhymed rewriting: {rhymed_text}
"""
    )
    print(f"\n\n{editor_notes}\n\n")

    return editor_notes.split("Final translation:")[1]

In [12]:
translate_in_stages(poem, language)



Translation:
Humpty Dumpty zat op een muur,
Humpty Dumpty had een grote val.
Alle paarden van de koning en alle mannen van de koning
konden Humpty niet weer in elkaar zetten.




Rewriting:
Humpty Dumpty zat op een hoge muur hoog,
En viel eraf met een grote klap zo droog.
De koning stuurde zijn paarden en mannen zo fijn,
Maar Humpty bleef gebroken, het kon niet meer lijn.




The original text is a well-known English nursery rhyme with a consistent rhyme
scheme and meter. The naive translation is a literal translation of the
original text, but it doesn't quite capture the same rhythm and rhyme scheme.
The rhymed rewriting attempts to improve upon this by using a more
natural-sounding Dutch rhyme scheme, but some of the wording and phrasing could
be improved for better clarity and flow.

The main issue with the rhymed rewriting is that it uses some forced rhymes and
wordings, such as "zo droog" and "zo fijn", which don't quite fit naturally
with the rest of the sentence. Additionally,

'\nHumpty Dumpty zat op een hoge muur hoog,\nEn viel eraf met een grote klap, een noodlottige val zo droog.\nDe koning stuurde zijn paarden en mannen zo snel,\nMaar Humpty bleef gebroken, en kon niet meer heel.'

## Task 3. Finding a hero

Our kingdom has a very formal process for approving a hero for a specific quest.
You task is to implement the approvement process using LLM calls:

You receive a request for a hero and a description of a hero to hire for this quest.

**Step 1**. Check that request is formally correct. You can come up with your own ideas, but we suggest the following criteria:
- It has a name of the person requesting a hero and a date;
- It has a description of who's going to supply the hero with money and other resources;
- It has a reason why the hero is needed, some quest or challenge;
- It has a recommended qualification for the hero;

**Step 2**. Check that the problem with which the request is trying to deal is sufficient to actually find a hero, or perhaps an author might do it themself or find an easier solution.

**Step 3**. Make sure that the description of the hero is compatible with the quest and requirements placed on the hero.

These steps should be performed sequentially, one after another. If any of the stages fail, immediately return a refusal with justification - you don't want to waste any more compute on unworthy queries! If all the three steps succeed, return "accepted".

In [13]:
def check_hero_request(request_for_a_hero: str, hero_descripition: str) -> str:
    pass

In [14]:
formal_checker_system_prompt = """
You are presented with a request to hire a hero, check the following formalities:
It has a name of the person requesting a hero and a date;
It has a description of who's going to supply the hero with money and resource;
It has a reason why the hero is needed, quest they are going to complete;
It has a recommended qualifications for the hero;
If any of those isn't true output the following:
REFUSE: (here some brief justification why)
If all of the criteria are good, output 'ACCEPT'
"""

problem_scale_checker = """
You are presented with a request to hire a hero.
Access whether the problem described in the request is worthy of looking for a hero to solve.
For example:
Slaying a dragon - good
Buying apples - bad
Playing trumpet - bad
Delivering sacred artifact - good.
In general the problem should be quiet epic to be good.
If it's not good output:
REFUSE: (here briefly justify why itn't not fitting)
If it's good output:
'ACCEPT'
"""


consistency_checker_prompt = """
You are presented with a request to hire a hero and the hero's description.
You should make sure that whatever the problem described in the request
can actually be solved by the hero proposed, based on the requirements in
the request and the qualities of the hero presented.
If the hero is fitting, output: 'ACCEPT'
Otherwise output:
REFUSE: (some brief justification why we reject the hero for this request)
"""


def check_hero_request(request_for_a_hero: str, hero_descripition: str) -> str:
    formal_check = answer_with_llm(
        system_prompt=formal_checker_system_prompt,
        prompt=request_for_a_hero
    )
    if "REFUSE" in formal_check:
        return False, formal_check


    problem_scale_check = answer_with_llm(
        system_prompt=problem_scale_checker,
        prompt=request_for_a_hero
    )
    if "REFUSE" in problem_scale_check:
        return False, problem_scale_check

    consistency_check = answer_with_llm(
        system_prompt=consistency_checker_prompt,
        prompt=f"Request: {request_for_a_hero}, hero_description: {hero_descripition}"
    )
    if "REFUSE" in consistency_check:
        return False, consistency_check

    return True, ''


In [15]:
epic_request = """
Epic Hero Request
Name of Requestor: Lord Aeldric of the Silver Vale
Date: The 15th Day of Bloomrise, Year 1025 of the Dawnstar Calendar

Resource Provision:
The hero shall be provisioned by the Guild of Eternal Flame, a conclave of wealthy artificers and arcane financiers, who have pledged a sum of 500,000 gold crowns, enchanted arms and armor, rare tomes of forgotten magic, a sky-bound griffon steed, and a personal aide skilled in healing and reconnaissance. All resources shall be delivered at the Hall of Summoning in Ironhold.

Purpose of Request / Quest Description:
Darkness stirs in the Hollow Spine Mountains, where the Obsidian Serpent — an ancient beast thought long dead — has risen anew. Villages lie in ruin, and the skies turn black with ash. The hero is summoned to descend into the Abyssal Breach, recover the lost Emberheart Crystal, and seal the rift before the World Spine fractures and all realms fall into chaos.

Recommended Hero Qualifications:

Proven mastery in combat, both arcane and martial

Experience in surviving extreme environments and demonic incursions

Wisdom enough to resist corruption, and strength to slay without hesitation

Familiarity with ancient dialects and lost technologies

A heart unwavering in the face of despair, and a spirit unbreakable by shadow

Let the stars guide the right soul to answer. The fate of the realms balances on a blade’s edge.
"""

not_epic_request = """
Epic Hero Request
Name of Requestor: Steve
Date: 3rd of January 2025

Resource Provision:
I'll pay from my pocket

Quest Description:
I need someone to run to a supermarket for me, i'm hungry

Required qualification:
- Be very fast
- Be smart to buy good snacks.
"""

wrong_request = """
Hello, I need a mighty warrior to slay evil, thank you!
"""

In [16]:
epic_hero = """
Hero Profile: Kaelen Thorne, the Ash-Wrought Sentinel

Forged in the fires of the Blistering Wars and tempered by years wandering the haunted ruins of the Old Kingdoms, Kaelen Thorne is a battle-scarred veteran clad in rune-etched obsidian armor. With one eye gifted by the Seers of Valemire—able to glimpse the truth behind illusions—and a blade forged from a fallen star, Kaelen walks the line between light and shadow.

Equal parts scholar and warrior, Kaelen speaks the tongues of forgotten realms and wields spells that twist the very air. Haunted but unyielding, Kaelen has turned away crowns and glory before—but for a quest that may decide the fate of all creation, the Sentinel rises once more.
"""

not_so_epic_hero = """
Tom the cat

Is a cat, supposed to catch mice, but can't really do it.
Has a lot of different surprising weapons and contraptions, but they always work against them.

Works for cat food.
"""

In [17]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=epic_hero)

(True, '')

In [18]:
check_hero_request(request_for_a_hero=epic_request, hero_descripition=not_so_epic_hero)

(False,
 "REFUSE: The hero proposed, Tom the cat, does not meet the qualifications\noutlined in the request. Tom's inability to catch mice, a relatively simple\ntask, suggests a lack of proven mastery in combat, and there is no indication\nthat Tom has experience with arcane combat or surviving extreme environments.\nAdditionally, Tom's contraptions working against them implies a lack of wisdom\nand strength, and there is no mention of Tom being familiar with ancient\ndialects or lost technologies. Overall, Tom's skills and characteristics do not\nalign with the demands of the quest to defeat the Obsidian Serpent and save the\nrealms.")

In [19]:
check_hero_request(request_for_a_hero=not_epic_request, hero_descripition=epic_hero)

(False,
 "REFUSE: The task of running to a supermarket to buy snacks is a mundane,\neveryday chore that doesn't require heroic efforts or exceptional abilities,\nmaking it unworthy of a heroic quest.")

In [20]:
check_hero_request(request_for_a_hero=wrong_request, hero_descripition=epic_hero)

(False,
 'REFUSE: The request lacks essential formalities, including the name of the\nperson requesting a hero, a date, a description of who will supply the hero\nwith money and resources, and recommended qualifications for the hero.')